In [1]:
import sys
sys.path.append('..')

In [2]:
from utils import Utils, AOC
from session import SESSION
import time
from collections import defaultdict
import re

In [3]:
aoc = AOC(session=SESSION)

In [4]:
aoc.verify_session()

True

In [5]:
aoc.get_today_file()

Local file found.


In [6]:
aoc.exraw = """Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1"""

## Contest part

In [7]:
def op_to_func(op):
    def apply(operator, first, second):
        match operator:
            case "*":
                return first * second
            case "+":
                return first + second
    
    f, operator, s = op.split(' ')

    def operation(x):
        first = f
        second = s
        if first == 'old':
            first = x
        else:
            first = int(first)
        if second == 'old':
            second = x
        else:
            second = int(second)

        return apply(operator, first, second)
    
    return operation

In [8]:
def op_to_func2(op):
    def operation(x):
        return eval(op.replace('old', str(x)))
    
    return operation

In [12]:
prod = Utils.product(map(int, re.findall(r'divisible by (\d+)', aoc.raw)))

In [13]:
class Monkey:
    def __init__(self, monkey, items_list, operation, test, true, false, monkies) -> None:
        self.id = int(monkey)
        self.items = [int(item) for item in items_list.split(', ')]
        self.inspect = op_to_func(operation)
        self.test = lambda x: not (x % int(test))
        self.true = int(true)
        self.false = int(false)
        self.monkies = monkies
        self.count = 0

    def process(self):
        while self.items:
            item = self.items.pop(0)
            self.count += 1
            if self.test(item := (self.inspect(item) % prod)):
                self.monkies[self.true].items.append(item)
            else:
                self.monkies[self.false].items.append(item)


In [14]:
monkies = []
for els in re.findall(r"""Monkey (\d+):
  Starting items: ([\d ,]+)
  Operation: new = (.+)
  Test: divisible by (\d+)
    If true: throw to monkey (\d+)
    If false: throw to monkey (\d+)""", aoc.raw):
    monkies.append(Monkey(*els, monkies))

In [15]:
for i in range(10000):
    for monkey in monkies:
        monkey.process()
    print(f'{i+1}/{10000} ({(i+1)/10000:.2%})', end='\r')

In [16]:
a, b = sorted([monkey.count for monkey in monkies], reverse=True)[:2]
a * b

14952185856